In [1]:
import numpy as np
import pandas as pd

from cobaya.run import run

from framework_pkg.framework import FrameWork
from framework_pkg.framework import bin_prediction, ideal_exposure

import warnings
warnings.filterwarnings('ignore')



Warning!: Python module pyslha not found, disabling slha reading routines


In [ ]:
param ={'SinT12'  : 0.308 ,
        'T13'  : 8.57, 
        'mu1'  : 0.,
        'mu2'  : 0., 
        'mu3'  : 0.  ,
        'mdm'  : 9.02e-4 , 
        'alpha': 0.  ,
        'eps' : 0.  ,
        'alpha_eps' : 0.  ,
        'M12'  : 6.9e-5 }


fw_spectrum  = FrameWork(threshold=3.5, bin_max=13, lat=36, efficiency_correction=False, resolution_correction=True)

spectrum_data = fw_spectrum.spectrum_data[:13]
energy_obs    = fw_spectrum.energy_obs

err_day = np.mean(spectrum_data[:,6:7],axis=1) / (spectrum_data[:,11] + spectrum_data[:,12])
data_day = spectrum_data[:,5] / (spectrum_data[:,11] + spectrum_data[:,12])

total_days = fw_spectrum.time_day
experiment_lat = fw_spectrum.lat

exposure = ideal_exposure(experiment_lat)

In [ ]:
def LogLikelihood(sint12, m12, mu2, eps):
    param['SinT12'] = sint12
    param['M12'] = m12 * 1e-5

    param['mu2'] = mu2
    param['alpha'] = 0.73
    param['eps'] = eps
    param['alpha_eps'] = 2.24

    param['mdm'] = 0.84 * 2e-4
    
    raw_result = fw_spectrum.__getitem__(param)

    alpha_day_spec, _, _ = bin_prediction(raw_result, exposure, total_days, spectrum=True)
    
    chi2 = np.sum((alpha_day_spec[0,:] - data_day)**2/err_day**2)

    return -0.5 * chi2

In [6]:
print(LogLikelihood(0.310, 6.9, 0., 0.))

print(LogLikelihood(0.310, 7.5, 0.11, 0.74))

-3.590782570860033
-6.175036225559259


In [5]:
info = {"likelihood": {"Chi2": LogLikelihood},
        
        "params": dict([("sint12" , {"prior" : {"dist": 'norm', "loc": 0.310, "scale" : 0.012},
                                  "latex" : r"\sin(\theta_{12})^2"  
                                  }),

                        
                        ("m12"    , {"prior" : {"dist": 'norm', "loc": 7.51, "scale": 0.19},
                                  "latex": r"\Delta m^2_{21} \ \rm 10^{-5} \ eV^2"  
                                  }), 
                        
                        ("mu2"    , {"prior" : {"min": 0, "max": 0.15},
                                   "latex" : r"\tilde{\mu}^2/2 "  
                                   }),
                        
                        # ("alpha"  , {"prior" : {"min": 0, "max": np.pi},
                        #            "latex" : r"\alpha "  
                        #            }),

                      
                        ("eps"  , {"prior" : {"min": 0, "max": 1.},
                                   "latex" : r"\epsilon "  
                                   })

                        # ("alpha_eps"  , {"prior" : {"min": 0, "max": np.pi},
                        #            "latex" : r"\alpha_\epsilon "  
                        #            }), 

                        # ("delta_p", {"prior" : {"min": -3, "max": 3},
                        #           "latex": r"\delta_p"  
                        #           }) 

                        # ("mdm", {"prior" : {"min": 1, "max": 10},
                        #            "latex": r"m_{\rm dm}"  
                        #           }) 
                        
                        ]),
        
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 100000 }},
        "output" : "output10/run_info"
       }

In [7]:
#updated_info,sampler = run(info)